<a href="https://colab.research.google.com/github/ksolosina/BCIT/blob/master/ml2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np #библиотека для работы с многомерными массивами данных и математическими операциями над ними
import pandas as pd #библиотека для анализа и обработки данных
pd.set_option('display.max.columns', 100)
import matplotlib.pyplot as plt #простое рисование графиков
import seaborn as sns #удобные дефолтные настройки графиков из matpotlib
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [16]:
import io
from google.colab import files
uploaded = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
data = pd.read_csv(io.BytesIO(uploaded['adult.data.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
data.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving adult.data.csv to adult.data.csv


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [17]:
# 1) Сколько мужчин и женщин ("sex" фича) в этом сете?
data["sex"].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

In [19]:
!python -m pip install --upgrade pip
!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.0.3)
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [21]:
uploaded = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
data = pd.read_csv(io.BytesIO(uploaded['googleplaystore.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
data.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving googleplaystore.csv to googleplaystore (4).csv


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [23]:
data.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [25]:
uploaded = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
data = pd.read_csv(io.BytesIO(uploaded['googleplaystore_user_reviews.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
data.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving googleplaystore_user_reviews.csv to googleplaystore_user_reviews.csv


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
